<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Archivo_de_descuentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

In [ ]:
!pip install xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import column_index_from_string, get_column_letter

In [ ]:
fecha_hoy = datetime.now().weekday()
if fecha_hoy == 0:
  fecha = str(datetime.strftime(datetime.now() - timedelta(2), "%d-%m-%Y"))
else:
  fecha = fecha = str(datetime.strftime(datetime.now() - timedelta(1), "%d-%m-%Y"))

archivo = "Reporte de descuentos " + fecha + ".xlsx"

# Archivos necesarios

In [ ]:
descuentos = pd.read_excel('Descuentos.xlsx', sheet_name=0, header = 9)
ventas = pd.read_excel('Ventas.xlsx', sheet_name=0, header = 5)
renglonaje = pd.read_excel("Renglonaje.xlsx", sheet_name=0)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Extraemos la columna Total y todas las formas de pago del archivo de ventas

In [ ]:
pagos = ventas.iloc[:,32:40]
pagos['Folio'] = ventas['Folio']
pagos = pagos.dropna()
pagos = pagos.reset_index(drop=True)

In [ ]:
try:
  ofertas = pd.read_excel('Ofertas.xlsx', sheet_name=0, header = 1)
  promos = ofertas[['Ean', 'Julio']]
  promos.rename(columns={'Ean':'EAN', 'Julio':'Distribuidor'}, inplace=True)
except:
  data = {'EAN':[-1.1], 'Distribuidor':[0]}
  promos = pd.DataFrame(data)


/tmp/ipython-input-25-528469239.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  promos.rename(columns={'Ean':'EAN', 'Julio':'Distribuidor'}, inplace=True)


Convertimos las columnas de total de ambos documentos en numérico y las unimos

In [ ]:
descuentos['Total'] = descuentos['Total'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df = descuentos.merge(pagos, on='Folio', how='left')

# Limpieza de datos

Limpiamos los nulos de las columnas

In [ ]:
df['Documento'] = df['Documento'].fillna(0)
df['Folio'] = df['Folio'].fillna(0)
df['Fecha'] = df['Fecha'].fillna(0)
df['Folio'] = df['Folio'].fillna(0)
df['Cliente'] = df['Cliente'].fillna(0)
df['Caja'] = df['Caja'].fillna(0)
df['Usuario'] = df['Usuario'].fillna(0)
df['Est'] = df['Est'].fillna(0)

El código generó duplicados por alguna razón (y no en todas las filas), así que los eliminamos

In [ ]:
df = df.drop_duplicates(df.loc[df['Folio'] != 0], keep='last')

Eliminamos las columnas que no tienen ningun valor

In [ ]:
lista = ['Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 23', 'Unnamed: 25', 'Unnamed: 28', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 14', 'Unnamed: 13', 'Unnamed: 18', 'Unnamed: 27', 'Unnamed: 34', 'Unnamed: 9']
df = df.drop(lista, axis=1)

Renombramos columnas que sí tienen datos importantes y limpiamos los nulos. Además, añadimos la columna de EAN

In [ ]:
df = df.rename(columns={'Unnamed: 6': 'Unidad', 'Unnamed: 10':'Producto', 'Unnamed: 11':"Clave", 'Unnamed: 12':'Descripción','Unnamed: 8':'Cantidad', 'Unnamed: 26':'Precio Unitario', 'Unnamed: 33':'Precio'})
df['Producto'] = df['Producto'].fillna(0)
df['Clave'] = df['Clave'].fillna(0)
df['Descripción'] = df['Descripción'].fillna(0)
df['Unidad'] = df['Unidad'].fillna(" ")

df.insert(8, 'EAN', 0)

Completamos los datos en las columnas de "Documento", "Fecha", "Cliente", etc.

In [ ]:
documento = df['Documento'][0]
for i in df.index:
  if df['Documento'][i] != 0:
    documento = df['Documento'][i]
  else:
    df.loc[i, 'Documento'] = documento
df.head()

,Documento,Fecha,Folio,Unidad,Cliente,Cantidad,Producto,Clave,EAN,Descripción,...,Precio,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-26 08:09:41,65283.0,,Público en General,NaN,0,0.0,0,0,...,NaN,25.0,$ 25.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
1,Ticket,0,0.0,PZA,0,1.0,"[7506240609834] 43827 Jaladera de acero, 128mm...",0.0,0,0,...,$ 25.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Ticket,2025-07-26 08:13:39,65284.0,,Público en General,NaN,0,0.0,0,0,...,NaN,75.0,$ 75.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
11,Ticket,0,0.0,$ 75.00,0,NaN,0,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Ticket,2025-07-26 08:17:32,65285.0,,Público en General,NaN,0,0.0,0,0,...,NaN,25.0,$ 25.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00


In [ ]:
fecha = df['Fecha'][0]
for i in df.index:
  if df['Fecha'][i] != 0:
    fecha = df['Fecha'][i]
  else:
    df.loc[i, 'Fecha'] = fecha

In [ ]:
cliente = df['Cliente'][0]
for i in df.index:
  if df['Cliente'][i] != 0:
    cliente = df['Cliente'][i]
  else:
    df.loc[i, 'Cliente'] = cliente

caja = df['Caja'][0]
for i in df.index:
  if df['Caja'][i] != 0:
    caja = df['Caja'][i]
  else:
    df.loc[i, 'Caja'] = caja

usuario = df['Usuario'][0]
for i in df.index:
  if df['Usuario'][i] != 0:
    usuario = df['Usuario'][i]
  else:
    df.loc[i, 'Usuario'] = usuario

est = df['Est'][0]
for i in df.index:
  if df['Est'][i] != 0:
    est = df['Est'][i]
  else:
    df.loc[i, 'Est'] = est

folio = df['Folio'][0]
for i in df.index:
  if df['Folio'][i] != 0:
    folio = df['Folio'][i]
  else:
    df.loc[i, 'Folio'] = folio

Extraemos el EAN, la clave y la descripción del producto

In [ ]:
for i in df.index:
  producto = df['Producto'][i]
  if producto != 0:
    a = producto.split(']')
    ean = a[0][1:]
    b = a[1]
    for j in range(len(b)):
      if b[j].isalpha():
        clave = b[1:j]
        descripcion = b[j:]
        break
    df.loc[i, 'EAN'] = ean
    df.loc[i, 'Clave'] = clave
    df.loc[i, 'Descripción'] = descripcion

df = df.drop('Producto', axis=1)

# df = df.merge(promos, on='EAN', how='left')

/tmp/ipython-input-34-2705896700.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7506240609834' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'EAN'] = ean
/tmp/ipython-input-34-2705896700.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '43827 ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Clave'] = clave


Limpiamos los datos de las columnas con datos financieros

In [ ]:
df['Efectivo'] = df["Efectivo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Cheque'] = df["Cheque"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Crédito'] = df["Crédito"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Transferencia'] = df["Transferencia"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Vales'] = df["Vales"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Tarjeta'] = df["Tarjeta"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Anticipo'] = df["Anticipo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['SICAR Pagos'] = df["SICAR Pagos"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

Eliminamos las filas que no tengan un producto asociado

In [ ]:
# Create a boolean mask for rows where 'Unidad' is one of the desired values
unidades = [' ', 'PZA', 'Set', 'PR', 'XFB', 'XRO', 'XKI', 'KG', 'KGM']
mask = df['Unidad'].isin(unidades)

# Drop rows where the 'Unidad' is NOT in the list of valid units
df = df.drop(df[~mask].index)

~mask: This is the inverse of the mask. Since mask is True for rows with valid units, ~mask is True for rows with invalid units.



In [ ]:
df['Precio Unitario'] = df['Precio Unitario'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Precio'] = df['Precio'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df = df.rename(columns={'Descuento $\n':'Descuento $'})
df['Descuento $'] = df['Descuento $'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
descuentos = df['Desc %'].str.strip().str.replace('%', '').astype(float)

Completamos la columna de "Total"

In [ ]:
df['Total'] = df['Total'].fillna(0)
df['Precio'] = df['Precio'].fillna(0)
df['Total'] = df['Total'] + df['Precio']

In [ ]:
df['Efectivo'] = df['Efectivo'].fillna(-1)
df['Cheque'] = df['Cheque'].fillna(-1)
df['Crédito'] = df['Crédito'].fillna(-1)
df['Transferencia'] = df['Transferencia'].fillna(-1)
df['Vales'] = df['Vales'].fillna(-1)
df['Tarjeta'] = df['Tarjeta'].fillna(-1)
df['Anticipo'] = df['Anticipo'].fillna(-1)
df['SICAR Pagos'] = df['SICAR Pagos'].fillna(-1)

Para evitar el ruido visual, las tendremos columnas vacías en los datos financieros que tengan 0 en su valor total

In [ ]:
efectivo = df['Efectivo'][0]
for i in df.index:
  if df['Efectivo'][i] != -1:
    if df['Efectivo'][i] == 0:
      efectivo = np.nan
    else:
      efectivo = 1
      df.loc[i, 'Efectivo'] = efectivo
  else:
    df.loc[i, 'Efectivo'] = efectivo

df['Efectivo'] = df['Efectivo'] * df['Total']

In [ ]:
cheque = df['Cheque'][0]
for i in df.index:
  if df['Cheque'][i] != -1:
    if df['Cheque'][i] == 0:
      cheque = np.nan
    else:
      cheque = 1
      df.loc[i, 'Cheque'] = cheque
  else:
    df.loc[i, 'Cheque'] = cheque

df['Cheque'] = df['Cheque'] * df['Total']

credito = df['Crédito'][0]
for i in df.index:
  if df['Crédito'][i] != -1:
    if df['Crédito'][i] == 0:
      credito = np.nan
    else:
      credito = 1
      df.loc[i, 'Crédito'] = credito
  else:
    df.loc[i, 'Crédito'] = credito

df['Crédito'] = df['Crédito'] * df['Total']

transferencia = df['Transferencia'][0]
for i in df.index:
  if df['Transferencia'][i] != -1:
    if df['Transferencia'][i] == 0:
      transferencia = np.nan
    else:
      transferencia = 1
      df.loc[i, 'Transferencia'] = transferencia
  else:
    df.loc[i, 'Transferencia'] = transferencia

df['Transferencia'] = df['Transferencia'] * df['Total']

vales = df['Vales'][0]
for i in df.index:
  if df['Vales'][i] != -1:
    if df['Vales'][i] == 0:
      vales = np.nan
    else:
      vales = 1
      df.loc[i, 'Vales'] = vales
  else:
    df.loc[i, 'Vales'] = vales

df['Vales'] = df['Vales'] * df['Total']

tarjeta = df['Tarjeta'][0]
for i in df.index:
  if df['Tarjeta'][i] != -1:
    if df['Tarjeta'][i] == 0:
      tarjeta = np.nan
    else:
      tarjeta = 1
      df.loc[i, 'Tarjeta'] = tarjeta
  else:
    df.loc[i, 'Tarjeta'] = tarjeta

df['Tarjeta'] = df['Tarjeta'] * df['Total']

anticipo = df['Anticipo'][0]
for i in df.index:
  if df['Anticipo'][i] != -1:
    if df['Anticipo'][i] == 0:
      anticipo = np.nan
    else:
      anticipo = 1
      df.loc[i, 'Anticipo'] = anticipo
  else:
    df.loc[i, 'Anticipo'] = anticipo

df['Anticipo'] = df['Anticipo'] * df['Total']

sicar_pagos = df['SICAR Pagos'][0]
for i in df.index:
  if df['SICAR Pagos'][i] != -1:
    if df['SICAR Pagos'][i] == 0:
      sicar_pagos = np.nan
    else:
      sicar_pagos = 1
      df.loc[i, 'SICAR Pagos'] = sicar_pagos
  else:
    df.loc[i, 'SICAR Pagos'] = sicar_pagos

df['SICAR Pagos'] = df['SICAR Pagos'] * df['Total']

Completamos la columna de descuentos

In [ ]:
df['Desc %'] = df['Desc %'].fillna(0)
descuento = df['Desc %'][0]
for i in df.index:
  if df['Desc %'][i] != 0:
    descuento = df['Desc %'][i]
  else:
    df.loc[i, 'Desc %'] = descuento

Le damos mejor formato a las columnas

In [ ]:
df['Unidad'] = df['Unidad'].replace(' ', '----------')
df['Cantidad'] = df['Cantidad'].fillna('----------')
df['Precio Unitario'] = df['Precio Unitario'].fillna('----------')
df['Precio'] = df['Precio'].replace(0, '----------')
df['Clave'] = df['Clave'].replace(0, '----------')
df['EAN'] = df['EAN'].replace(0, '----------')
df['Descripción'] = df['Descripción'].replace(0, '----------')

Eliminamos las filas sin datos relevantes

In [ ]:
df = df.drop(df.tail(5).index)

Separamos el dato de "Fecha" en Fecha y Hora

In [ ]:
df.insert(1, "Día", "")
df.insert(2, "Hora", "")

In [ ]:
df['Fecha'] = df['Fecha'].astype(str)

In [ ]:
for i in df.index:
  fecha = df['Fecha'][i].split(' ')
  df.loc[i, 'Día'] = fecha[0]
  df.loc[i, 'Hora'] = fecha[1]
df.drop('Fecha', axis=1, inplace=True)
df.rename(columns={'Día':'Fecha'}, inplace=True)

# Hoja de reporte

Extraemos los datos relevantes del dataframe principal

In [ ]:
descuentos = df[['Folio', 'Cliente','Clave', 'EAN', 'Desc %', 'Precio Unitario', 'Cantidad', 'Precio']]

Filtramos los productos

Nota: Algunas veces, el EAN que aparece en el archivo de SICAR no es numérico. La parte de *errors = coerce* pone en NaN esos EAN

In [ ]:
descuentos = descuentos.loc[descuentos['Clave'] != '----------']
descuentos['Clave'] = descuentos['Clave'].replace('', 0)
descuentos['EAN'] = pd.to_numeric(descuentos['EAN'], errors='coerce')
descuentos = descuentos.reset_index(drop=True)

Extraemos los datos de renglonaje, de la hoja del catálogo de Truper

In [ ]:
costos = renglonaje[['ean', 'precio mayoreo con IVA', 'precio mínimo de venta', 'precio público con IVA']]
costos = costos.rename(columns={'ean':'EAN'})

In [ ]:
costos['precio mayoreo con IVA'] = costos['precio mayoreo con IVA'].replace('*', 0)
costos['EAN'] = costos['EAN'].fillna(0)
costos['EAN'] = costos['EAN'].replace('', 0)
costos['EAN'] = pd.to_numeric(costos['EAN'], errors='coerce')

/tmp/ipython-input-51-4086224631.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  costos['precio mayoreo con IVA'] = costos['precio mayoreo con IVA'].replace('*', 0)


In [ ]:
descuentos = descuentos.merge(costos, on='EAN', how='left')
descuentos = descuentos.merge(promos, on='EAN', how='left')

In [ ]:
descuentos['Diferencia mayoreo'] = descuentos['Precio Unitario'] - descuentos['precio mayoreo con IVA']
descuentos['Diferencia minimo'] = descuentos['Precio Unitario'] - descuentos['precio mínimo de venta']
descuentos['Diferencia publico'] = descuentos['Precio Unitario'] - descuentos['precio público con IVA']
descuentos['Diferencia dist.'] = descuentos['Precio Unitario'] - descuentos['Distribuidor']

Creación del excel

In [ ]:
writer = pd.ExcelWriter(archivo, engine="xlsxwriter")

In [ ]:
df.to_excel(writer, sheet_name='SICAR', index=False)
descuentos.to_excel(writer, sheet_name='Reporte', index=False)

In [ ]:
writer.close()

# Dando formato al Excel

## Hoja de SICAR

In [ ]:
wb = load_workbook(archivo)
ws = wb['SICAR']

## Columnas comúnmente usadas

In [ ]:
last_col = column_index_from_string('P')
col_z = column_index_from_string('Z')
col_dif_mayoreo = column_index_from_string('M')
col_dif_min = column_index_from_string('N')
col_dif_pub = column_index_from_string('O')
col_dif_dist = column_index_from_string('P')

### Fuentes
- font_t = Fuente para la fila de metadatos
- font = Fuente para el resto de documentos

In [ ]:
font_t = Font(name='Comfortaa', size=12, bold=True, italic=False, color='ffffff')
font = Font(name='Comfortaa', size=11, bold=False, italic=False, color='000000')

### Colores de fondo
- fill_b = negro
- fill_r = rojo
- fill_g = verde
- fill_y = amarillo
- fill_p = morado
- fill_bl = azul

In [ ]:
fill_b = PatternFill(fill_type="solid", start_color="000000", end_color="000000")
fill_r = PatternFill(fill_type="solid", start_color="ffc7ce", end_color="ffc7ce")
fill_g = PatternFill(fill_type="solid", start_color="b6d7a8", end_color="b6d7a8")
fill_y = PatternFill(fill_type="solid", start_color="ffe599", end_color="ffe599")
fill_p = PatternFill(fill_type="solid", start_color="b4a7d6", end_color="b4a7d6")
fill_bl = PatternFill(fill_type="solid", start_color="a4c2f4", end_color="a4c2f4")

In [ ]:
for col in range(1,col_z + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

ws.column_dimensions['F'].width = 40

In [ ]:
for col in range(1, 27):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='center')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, 27):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='center')

for row in range(2, 1000):
  if ws.cell(row=row, column=18).value == None:
    break
  ws.cell(row=row, column=18).fill = fill_g

In [ ]:
num_form = '$ #,##0.00'
for row in range(2, 1000):
  for col in range(15, 28):
    ws.cell(row=row, column=col).number_format = num_form

In [ ]:
wb.save(archivo)

## Hoja de Reporte

In [ ]:
ws = wb['Reporte']

In [ ]:
for col in range(1, last_col + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

In [ ]:
for col in range(1, last_col + 1):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='right')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, last_col + 1):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='right')

### Explicación del ciclo for:


Por cada fila desde la 2 hasta la 1000:

---

- SI no hay número de folio, ROMPE CICLO
- SI el valor de diferencia entre el precio y el precio mínimo de venta es nulo, pinta de morado; SI NO -> SI la diferencia con el mayoreo y la diferencia con el mínimo de venta es diferente de 0 -> A

---

A

- SI el valor de descuento es 0.00% -> B; SI NO -> Pinta de amarillo toda la fila

---

B

- SI la diferencia con el precio distribuidor es diferente de 0, pinta de rojo; SI NO, pinta de azul

---
Los colores significan lo siguiente:
---
- El color rojo representa que se dió a un precio menor al mínimo SIN descuento o promoción.

- El color amarillo significa que se dió un precio menor al mínimo CON descuento.

- El color morado representa un error o una fila atípica.

- El azul representa que se vendió un producto con oferta/promoción

In [ ]:
for row in range(2, 1000):
  if(ws.cell(row=row, column=1).value == None):
    break

  dif_may = ws.cell(row=row, column=col_dif_mayoreo).value
  dif_min = ws.cell(row=row, column=col_dif_min).value
  dif_pub = ws.cell(row=row, column = col_dif_pub).value
  dif_dist = ws.cell(row=row, column=col_dif_dist).value
  descuento = ws.cell(row=row, column=5).value

  if(dif_min == None):
    for col in range(1, last_col + 1):
      ws.cell(row=row, column=col).fill = fill_p
  elif(dif_may != 0 and dif_min != 0 and dif_pub != 0):
    if(descuento == '0.00%'):
      if(dif_dist != 0):
        for col in range(1, last_col + 1):
          ws.cell(row=row, column=col).fill = fill_r
      else:
        for col in range(1, last_col + 1):
          ws.cell(row=row, column=col).fill = fill_bl
    else:
      for col in range(1, last_col + 1):
        ws.cell(row=row, column=col).fill = fill_y

Le da formato de número a las filas de datos financieros

In [ ]:
num_form = '$ #,##0.00'
for row in range(2, 1000):
  ws.cell(row=row, column=6).number_format = num_form
  for col in range(8, last_col + 1):
    ws.cell(row=row, column=col).number_format = num_form

In [ ]:
wb.save(archivo)